# 02 - Exploratory Analysis (SQL)

## Analytical Objective

With the cleaned dataset stored in a SQL database, this notebook focuses on structured exploratory analysis using SQL queries.

The objective is to quantify customer churn behavior, measure its financial impact, and identify high-risk customer segments that represent the greatest business opportunity for retention strategies.

---

## Business Questions

This analysis aims to answer the following key questions:

1. What is the overall churn rate?
2. What percentage of monthly revenue is lost due to churn?
3. Do churned customers generate higher or lower revenue on average?
4. Which contract types show the highest churn?
5. How does churn vary across customer tenure segments?
6. Is churn concentrated in specific high-impact segments?

---

## Methodology

All analytical queries in this notebook are executed using SQL against the structured database created in the data preparation stage.

The analysis follows a top-down approach:

1. Overall churn and revenue impact  
2. Revenue distribution between churned and retained customers  
3. Segmentation analysis by contract type  
4. Segmentation analysis by tenure  
5. Identification of priority retention segments  

The goal is not only to compute metrics but to understand their business implications.


### Database Connection

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../sql/telco_churn.db")

# check
pd.read_sql("SELECT * FROM telco", conn)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


### Overall Churn Rate

In [2]:
query = """
SELECT 
    COUNT(*) AS total_customers,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS churned_customers,
    ROUND(
        100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*),
        2
    ) AS churn_rate_percent
FROM telco;
"""

pd.read_sql(query, conn)

,total_customers,churned_customers,churn_rate_percent
0,7043,1869,26.54


The overall churn rate indicates that a significant portion of customers have discontinued their service. A churn rate above 25% is generally considered high in subscription-based businesses and represents a major revenue sustainability risk.

This suggests that customer retention should be treated as a strategic priority, particularly if churn is concentrated within specific segments.

### Revenue Impact

In [3]:
query = """
SELECT 
    ROUND(SUM(MonthlyCharges), 2) AS total_monthly_revenue,
    ROUND(
        SUM(CASE WHEN Churn = 'Yes' THEN MonthlyCharges ELSE 0 END),
        2
    ) AS churned_monthly_revenue,
    ROUND(
        100.0 * SUM(CASE WHEN Churn = 'Yes' THEN MonthlyCharges ELSE 0 END) 
        / SUM(MonthlyCharges),
        2
    ) AS revenue_at_risk_percent
FROM telco;
"""

pd.read_sql(query, conn)

,total_monthly_revenue,churned_monthly_revenue,revenue_at_risk_percent
0,456116.6,139130.85,30.5


Although churn rate measures customer loss, revenue churn quantifies its financial impact. 

The results show that churned customers account for a disproportionately large share of total monthly revenue. This implies that the company is not only losing customers, but also losing higher-value customers, amplifying the economic impact.

Revenue churn exceeding customer churn is a strong signal of structural retention issues.

### ARPU Comparison

In [4]:
query = """
SELECT 
    Churn,
    ROUND(AVG(MonthlyCharges), 2) AS avg_monthly_charge,
    COUNT(*) AS customers
FROM telco
GROUP BY Churn;
"""

pd.read_sql(query, conn)

,Churn,avg_monthly_charge,customers
0,No,61.27,5174
1,Yes,74.44,1869


Churned customers exhibit a higher average monthly charge compared to retained customers. This indicates that customers generating more revenue are also more likely to leave.

This pattern may suggest pricing sensitivity, contract structure issues, or dissatisfaction among higher-paying customers. From a business perspective, this represents a high-risk scenario, as customer acquisition costs are unlikely to offset the loss of higher-value subscribers.

### Churn by Contract Type

In [5]:
query = """
SELECT 
    Contract,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS churned_customers,
    ROUND(
        100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*),
        2
    ) AS churn_rate_percent,
    ROUND(AVG(MonthlyCharges), 2) AS avg_monthly_charge
FROM telco
GROUP BY Contract
ORDER BY churn_rate_percent DESC;
"""

pd.read_sql(query, conn)

,Contract,total_customers,churned_customers,churn_rate_percent,avg_monthly_charge
0,Month-to-month,3875,1655,42.71,66.40
1,One year,1473,166,11.27,65.05
2,Two year,1695,48,2.83,60.77


Churn is heavily concentrated among customers with month-to-month contracts, while long-term contracts (one-year and two-year) show significantly lower churn rates.

This suggests that contract duration plays a critical role in customer retention. Customers without long-term commitment appear more likely to switch providers.

Encouraging migration from month-to-month to longer-term contracts could be a highly effective retention strategy.

### Churn by Tenure Segment

In [6]:
query = """
SELECT 
    CASE 
        WHEN tenure <= 12 THEN '0-1 year'
        WHEN tenure <= 24 THEN '1-2 years'
        WHEN tenure <= 48 THEN '2-4 years'
        ELSE '4+ years'
    END AS tenure_group,
    COUNT(*) AS total_customers,
    ROUND(
        100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*),
        2
    ) AS churn_rate_percent
FROM telco
GROUP BY tenure_group
ORDER BY churn_rate_percent DESC;
"""

pd.read_sql(query, conn)

,tenure_group,total_customers,churn_rate_percent
0,0-1 year,2186,47.44
1,1-2 years,1024,28.71
2,2-4 years,1594,20.39
3,4+ years,2239,9.51


Churn is significantly higher among newer customers, particularly those with less than one year of tenure.

This indicates that churn is primarily an early-lifecycle problem. Customers who remain beyond the first year show progressively lower churn rates, suggesting that long-term engagement improves retention stability.

Improving onboarding experience and early-stage customer satisfaction may significantly reduce overall churn.

### High-Impact Segment Identification

In [7]:
query = """
SELECT 
    COUNT(*) AS customers,
    ROUND(SUM(MonthlyCharges), 2) AS monthly_revenue,
    ROUND(AVG(MonthlyCharges), 2) AS avg_monthly_charge
FROM telco
WHERE Contract = 'Month-to-month'
AND tenure <= 12;
"""

pd.read_sql(query, conn)

,customers,monthly_revenue,avg_monthly_charge
0,1994,116086.5,58.22


In [8]:
query = """
WITH total_churn AS (
    SELECT COUNT(*) AS total_churned
    FROM telco
    WHERE Churn = 'Yes'
),
segment_churn AS (
    SELECT COUNT(*) AS segment_churned
    FROM telco
    WHERE Churn = 'Yes'
    AND Contract = 'Month-to-month'
    AND tenure <= 12
)

SELECT 
    segment_churned,
    total_churned,
    ROUND(100.0 * segment_churned / total_churned, 2) AS percent_of_total_churn
FROM total_churn, segment_churn;
"""

pd.read_sql(query, conn)

,segment_churned,total_churned,percent_of_total_churn
0,1024,1869,54.79


The segment consisting of customers under month-to-month contracts with tenure of one year or less represents a disproportionately large share of total churn while accounting for a smaller fraction of overall revenue.

Although this segment represents only 25% of monthly revenue, it accounts for nearly 55% of total churn. This indicates a highly concentrated retention problem among new customers under month-to-month contracts.

This concentration of churn risk identifies a high-impact retention opportunity. Targeted interventions focused on this segment could produce outsized improvements in overall retention performance.

Strategically, this segment should be prioritized for retention campaigns, contract migration incentives, and onboarding optimization initiatives.